In [1]:
from openai import OpenAI
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

client = OpenAI()


In [4]:
response = client.responses.create(
    model="gpt-4.1-mini",
    tools=[{"type": "web_search"}],
    input="2025년 전 세계에서 있었던 좋은 뉴스 하나 알려줘."
)

print(response.output_text)
print(response.output[0])

2025년에는 전 세계적으로 여러 긍정적인 소식이 있었습니다. 그 중 하나는 2025년 5월에 열린 제78회 칸 영화제에서 한국 영화 '기생충'이 황금종려상을 수상한 것입니다. 이로 인해 한국 영화의 국제적 위상이 높아졌으며, 세계 영화 산업에 큰 영향을 미쳤습니다. 
ResponseFunctionWebSearch(id='ws_016ea2ee04fa226b006954d41854cc81a08519159f24660740', action=ActionSearch(query='좋은 뉴스 2025년 전 세계', type='search', sources=None), status='completed', type='web_search_call')


In [ ]:
client.responses.create(
    model="gpt-4.1",
    input="우리 제품 환불 규정 요약해줘.",
    tools=[{
        "type": "file_search",
        "vector_store_ids": ["<vector_store_id>"],
    }],
)

In [5]:
import base64

response = client.responses.create(
    model="gpt-4.1-mini",
    input="다 먹은 제철 감자칩 그림 그려줘.",
    tools=[{"type": "image_generation"}],
)

# Save the image to a file
image_data = [
    output.result
    for output in response.output
    if output.type == "image_generation_call"
]

if image_data:
    image_base64 = image_data[0]
    with open("pic.png", "wb") as f:
        f.write(base64.b64decode(image_base64))

In [10]:
instructions = """
You are a personal math tutor. When asked a math question,
write and run code using the python tool to answer the question.
"""

resp = client.responses.create(
    model="gpt-4.1-mini",
    tools=[
        {
            "type": "code_interpreter",
            "container": {"type": "auto", "memory_limit": "4g"}
        }
    ],
    instructions=instructions,
    input="I need to solve the equation 3x + 11 = 14. Can you help me?",
)

print(resp.output)

for output in resp.output:
  if output.type =="message":
    print(output.content[0].text)

[ResponseOutputMessage(id='msg_059f179410629443006954d7d6871481a09be5a6e19dab3e31', content=[ResponseOutputText(annotations=[], text="Sure! To solve the equation \\(3x + 11 = 14\\), we need to isolate the variable \\(x\\). Let's solve this step-by-step.", type='output_text', logprobs=[])], role='assistant', status='completed', type='message'), ResponseCodeInterpreterToolCall(id='ci_059f179410629443006954d7d7b36881a099fddb2c09345351', code="from sympy import symbols, Eq, solve\n\n# Define the variable and equation\nx = symbols('x')\nequation = Eq(3*x + 11, 14)\n\n# Solve the equation for x\nsolution = solve(equation, x)\nsolution", container_id='cntr_6954d7d52be8819190f81d507d6b3b8b0cc14c22f4ac1dc6', outputs=None, status='completed', type='code_interpreter_call'), ResponseOutputMessage(id='msg_059f179410629443006954d7db3f2881a08bfb55b4ea133cd2', content=[ResponseOutputText(annotations=[], text='The solution to the equation \\(3x + 11 = 14\\) is \\(x = 1\\). If you have any other questio

In [14]:
tools = [
    {
        "type": "function",
        "name": "get_order_status",
        "description": "주문 ID를 받아 현재 배송 상태를 조회한다.",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": {"type": "string"},
            },
            "required": ["order_id"],
        },
    },
    {
        "type": "function",
        "name": "create_support_ticket",
        "description": "고객 문의/불만을 티켓으로 생성한다.",
        "parameters": {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "description": {"type": "string"},
                "priority": {
                    "type": "string",
                    "enum": ["low", "normal", "high"],
                },
            },
            "required": ["title", "description"],
        },
    },
]


In [15]:
def get_order_status(order_id: str):
    return {"status": "shipped"}

def create_support_ticket(title: str, description: str, priority: str):
    return {"ticket_id": "12345", "priority" : priority}

In [19]:
import json

def handle_tool_call(item):
    args = json.loads(item.arguments)
    print(args)
    if item.name == "get_order_status":
        return get_order_status(**args)
    elif item.name == "create_support_ticket":
        return create_support_ticket(**args)
    else:
        return {"error": f"Unknown tool {item.name}"}



def run_support_agent(user_message: str):
    # 1차 호출: 어떤 툴을 쓸지 모델에게 맡김
    messages = [{"role": "user", "content": user_message}]
    resp = client.responses.create(
        model="gpt-4.1-mini",
        tools=tools,
        instructions = "사용자 요청에 대해 적절한 도구를 선택하되, 불만/요청 등의 문구가 있으면 create_support_ticket 도구를 사용해",
        input=messages,
    )

    messages += list(resp.output)  # function_call 들을 히스토리에 추가

    # 여러 function_call에 대해 반복
    for item in resp.output:
        if item.type == "function_call":
            print(item)
            result = handle_tool_call(item)
            messages.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps(result, ensure_ascii=False),
            })

    # 최종 답변 생성
    resp2 = client.responses.create(
        model="gpt-4.1-mini",
        tools=tools,
        input=messages,
        instructions=(
            "너는 고객센터 상담사야. function_call_output으로 전달된 "
            "주문/티켓 정보를 활용해서, 상황을 정리해주고 다음 액션을 제안해줘.티켓이 생성된 경우 티켓 번호를 알려줘야 해."
        ),
    )
    print(resp2.output_text)




In [16]:
run_support_agent("주문 2024-000123 상태 좀 알려줘.")

ResponseFunctionToolCall(arguments='{"order_id":"2024-000123"}', call_id='call_Ii7mhJqn3BBz9I8iZnkRzZ8i', name='get_order_status', type='function_call', id='fc_075b76d015d32aff006954dba096548192ad148020d0ded37f', status='completed')
{'order_id': '2024-000123'}
주문 번호 2024-000123은 현재 배송 중 상태입니다. 추가로 궁금한 점이나 문제가 있으신가요? 도움이 필요하시면 말씀해 주세요.


In [20]:
# run_support_agent("앱이 자꾸 튕겨서 불만 접수 티켓 만들어줘.")
run_support_agent("앱 배경 색이 좀 거슬리는데 수정 요청하고 싶어.")

ResponseFunctionToolCall(arguments='{"title":"앱 배경 색상 변경 요청","description":"사용자가 앱 배경 색상이 거슬린다고 하여 배경 색상 수정 요청을 하였습니다.","priority":"normal"}', call_id='call_6cYBbTxgctHFRwXZzSsxBOot', name='create_support_ticket', type='function_call', id='fc_02fb675de1f57c5e006954dcb390b0819d9ea2c4c42c9f70bf', status='completed')
{'title': '앱 배경 색상 변경 요청', 'description': '사용자가 앱 배경 색상이 거슬린다고 하여 배경 색상 수정 요청을 하였습니다.', 'priority': 'normal'}
앱 배경 색상에 대해 수정 요청 티켓이 생성되었습니다. 티켓 번호는 12345번입니다. 요청하신 내용은 개발팀에 전달되어 검토될 예정입니다. 추가로 다른 불편한 점이나 요청사항이 있으면 알려주세요.


In [21]:
run_support_agent("주문 2024-000123 너무 늦게 오고, 환불 불만 티켓도 같이 남겨줘.")

ResponseFunctionToolCall(arguments='{"order_id":"2024-000123"}', call_id='call_OxNk8sshYD7HQ64A6CWeewIL', name='get_order_status', type='function_call', id='fc_0357b47e562e3ad8006954dd2aec50819da2391d0fff3e8576', status='completed')
{'order_id': '2024-000123'}
ResponseFunctionToolCall(arguments='{"title":"주문 2024-000123 배송 지연 및 환불 불만","description":"주문 2024-000123이 너무 늦게 도착하여 환불을 요청하며 이에 대한 불만 접수","priority":"high"}', call_id='call_7TGOw8ICjY0mpoo7quF65MBR', name='create_support_ticket', type='function_call', id='fc_0357b47e562e3ad8006954dd2b2a54819d9eb048add1e54f22', status='completed')
{'title': '주문 2024-000123 배송 지연 및 환불 불만', 'description': '주문 2024-000123이 너무 늦게 도착하여 환불을 요청하며 이에 대한 불만 접수', 'priority': 'high'}
주문 번호 2024-000123은 현재 배송 상태가 '배송됨(shipped)'으로 확인되었습니다. 하지만 배송이 늦어진 점에 대해 불편을 드려 죄송합니다.

고객님의 환불 요청과 배송 지연 관련 불만 건으로 티켓(번호: 12345)을 생성했습니다. 이 티켓은 우선 순위가 높게 처리될 예정이며, 문제 해결을 위해 곧 연락을 드리겠습니다.

추가로 궁금한 점이나 요청 사항이 있으면 언제든 말씀해 주세요.


In [23]:
mixed_tools = [
    {"type": "web_search"},
    {
        "type": "function",
        "name": "save_insight",
        "description": "중요한 리서치 인사이트를 내부 시스템에 저장한다.",
        "parameters": {
            "type": "object",
            "properties": {
                "topic": {"type": "string"},
                "sentiment": {"type": "string", "enum": ["positive", "negative", "neutral"]},
                "summary": {"type": "string"},
            },
            "required": ["topic", "sentiment", "summary"],
        },
    },
]

In [25]:
def fake_save_insight(topic, sentiment, summary):
    print(f"[저장됨] topic={topic}, sentiment={sentiment}")
    print(f"summary={summary[:80]}...")
    return {"status": "ok"}

def handle_mixed_tool_call(item):
    import json
    args = json.loads(item.arguments)
    if item.name == "save_insight":
        return fake_save_insight(**args)
    else:
        return {"error": f"unknown tool {item.name}"}

def run_research_agent(question: str):
    messages = [{"role": "user", "content": question}]
    resp = client.responses.create(
        model="gpt-5",
        tools=mixed_tools,
        input=messages,
    )
    messages += list(resp.output)

    # function_call만 우리가 처리, web_search는 OpenAI 쪽에서 이미 처리 완료
    for item in resp.output:
        if item.type == "function_call":
            print(item)
            result = handle_mixed_tool_call(item)

            messages.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps(result, ensure_ascii=False),
            })

    resp2 = client.responses.create(
        model="gpt-5",
        tools=mixed_tools,
        input=messages,
        instructions=(
            "웹 검색과 save_insight 함수를 적절히 사용해서 "
            "사용자 질문에 답하고, 중요한 인사이트는 저장해라."
        ),
    )
    print(resp2.output_text)

run_research_agent(" Apple 관련 긍정적인 뉴스 하나를 웹에서 찾아서 요약하고, 인사이트도 저장해줘.")

ResponseFunctionToolCall(arguments='{"topic":"Apple FY2025 Q4 실적(서비스 사상 최고·iPhone 9월 분기 신기록)","sentiment":"positive","summary":"애플이 2025회계연도 4분기(9/27 마감)에 매출 1,025억 달러(+8% YoY), EPS +13%를 발표했고, 서비스 매출은 사상 최고, iPhone 매출은 9월 분기 신기록을 기록. 경영진은 연말(12월 분기)에도 사상 최대 실적 가능성을 시사. 인사이트: 서비스 비중 확대와 iPhone 17 초기 수요가 견조하며, Apple Intelligence 확산이 생태계 잠금효과와 ARPU 상승에 기여할 가능성."}', call_id='call_4agfcy5cAJnV86Kt62ylnW3g', name='save_insight', type='function_call', id='fc_0a047bafb13a7a1a006954ded97fb88194888cddf4033495d1', status='completed')
[저장됨] topic=Apple FY2025 Q4 실적(서비스 사상 최고·iPhone 9월 분기 신기록), sentiment=positive
summary=애플이 2025회계연도 4분기(9/27 마감)에 매출 1,025억 달러(+8% YoY), EPS +13%를 발표했고, 서비스 매출은 사상 최고,...
다음은 최근의 애플 관련 ‘긍정적’ 뉴스 1건 요약입니다.

- 뉴스: 애플이 2025 회계연도 4분기(2025년 10월 30일 발표) 실적에서 매출 1,025억 달러로 전년 대비 8% 증가, 주당순이익(EPS) 13% 증가를 기록. 서비스 매출은 사상 최고, iPhone 매출은 9월 분기 기준 신기록을 달성. 애플은 연말(12월 분기) 전망도 낙관적으로 제시했습니다. citeturn0search1turn0search6

한줄 인사이트
- 서비스 비중 확대와 견조한 iPhone 수요가 복합적으로 실적을 견인하고 있어, 연말